In [1]:
import pandas as pd
import gc
from google.colab import drive
drive.mount('/content/drive')

data_root = '/content/drive/MyDrive/부트캠프/'
data_path = data_root + '11_파이널/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df3 = pd.read_parquet(data_path + 'data/train00.parquet')
# df3 = df3.drop('Unnamed: 0', axis=1, inplace=True)
# df3.to_parquet(data_path + 'data/train0.parquet')
# df3.info()
df3.head()

,남녀구분코드,연령,Segment,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,소지카드수_이용가능_신용,입회경과개월수_신용,...,연체입금원금_B0M,정상청구원금_B5M,선입금원금_B5M,정상입금원금_B5M,연체입금원금_B5M,연속무실적개월수_기본_24M_카드,연속유실적개월수_기본_24M_카드,신청건수_ATM_CA_B0,신청건수_ATM_CA_R6M,이용금액대
0,1,2,D,1,1,0,1,1,1,67,...,8104,14958,0,9205,5752,0,13,0,11,10000
1,0,1,E,1,1,1,1,1,1,12,...,826,3367,0,2546,821,0,12,0,0,3000
2,0,1,C,1,1,0,1,1,1,124,...,9364,23963,0,16949,7014,0,8,0,0,10000
3,1,2,D,1,1,0,1,2,2,27,...,10923,19614,0,8418,11196,0,5,0,0,10000
4,1,2,E,1,1,1,1,1,1,2,...,0,0,0,0,0,5,0,0,0,0


In [3]:
from imblearn.over_sampling import SMOTE

smote=SMOTE(sampling_strategy='not majority')

In [4]:
# df3 = df3.loc[:400000,:]
x = df3.drop(['Segment'], axis=1,)
# '대표청구서수령지구분코드', '청구서수령방법', '할인건수_R3M', '할인건수_B0M'는 encoding 문제로 제거
y = df3['Segment']

In [3]:
df3 = df3.loc[:400000,:]
x = df3.drop(['ID','기준년월','대표청구서수령지구분코드', '청구서수령방법', '할인건수_R3M', '할인건수_B0M'], axis=1,)

In [5]:
x.to_parquet(data_path + 'data/train00.parquet')

In [5]:
del df3
gc.collect()

0

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)

In [6]:
train_x,train_y=smote.fit_resample(X_train,y_train)
train_y.value_counts()

,count
Segment,
E,256273
D,256273
C,256273
A,256273
B,256273


In [7]:
from sklearn.preprocessing import LabelEncoder
# 타깃 라벨 인코딩
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(train_y)

In [8]:
import xgboost as xgb
from sklearn.metrics import f1_score

In [9]:
model = xgb.XGBClassifier(
    random_state=42
)
model.fit(train_x, y_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [10]:
y_pred = model.predict(X_test)
# f1-micro 평가
y_test_encoded = le_target.transform(y_test)
f1_micro = f1_score(y_test_encoded, y_pred, average='micro')
print(f"F1 score (micro): {f1_micro:.4f}")

F1 score (micro): 0.9032


In [11]:
import pickle
with open(data_path + 'data/01_model.dat', 'wb') as fp:
    pickle.dump(model, fp)
    pickle.dump(le_target, fp)


In [12]:
with open(data_path + 'data/01_model.dat', 'rb') as fp:
    test_model = pickle.load(fp)
    test_le_target = pickle.load(fp)
r1 = test_model.predict(X_test)
f1_macro = f1_score(y_test_encoded, r1, average='macro')
print(f"F1 score (macro): {f1_macro:.4f}")

F1 score (macro): 0.5455


In [13]:
f1_macro = f1_score(y_test_encoded, r1, average='micro')
print(f"F1 score (micro): {f1_macro:.4f}")

F1 score (micro): 0.9032


In [15]:
f1_weighted = f1_score(y_test_encoded, r1, average='weighted')
print(f"F1 score (micro): {f1_weighted:.4f}")

F1 score (micro): 0.9022
